<a href="https://colab.research.google.com/github/hadiroudsari/wine-detection/blob/master/wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wine prediction
 this project is prepered for bigdata course project, I have done some task on this specific dataset such as wine prediction...


---



here i load JDK 8 LTS


---



In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64" # Set path to JAVA_HOME


## **1.** Install PySpark and related dependencies

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

     |████████████████████████████████| 204.7MB 62kB/s 
     |████████████████████████████████| 204kB 41.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=aae5ff09c2a1cab8c45db3ca369c84f8ac8cfe533093e9ec648e1063497a2be8
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

## **2.** Import useful PySpark packages *italicized text*

In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf



# **3.** Create Spark context

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# **4.** Create <code>ngrok</code> tunnel to check the Spark UI

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-06-16 22:36:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.231.196.161, 54.152.45.100, 3.223.249.113, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.231.196.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.53MB/s    in 2.0s    

2020-06-16 22:36:15 (6.53 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [6]:
spark

In [7]:
sc._conf.getAll()

[('spark.driver.memory', '45G'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '39355'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.ui.port', '4050'),
 ('spark.driver.host', '369d245080c1'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1592346966691'),
 ('spark.executor.memory', '4G'),
 ('spark.ui.showConsoleProgress', 'true')]

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## **1. Link Google Colab to our Google Drive**

In [11]:
GDRIVE_DIR = "/content/gdrive" # Your own mount point on Google Drive
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive" # Your own home directory
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/1898985/dataSet" # Your own data directory

## **1. Link Google Colab to our Google Drive**

In [12]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/gdrive


## **2. Data Acquisition**

### **Download dataset file from URL directly to our Google Drive**

### **Read dataset file into a Spark Dataframe**


In [14]:
wine_df = spark.read.load(GDRIVE_DATA_DIR, 
                           format="csv", 
                           sep=",", 
                           inferSchema="true", 
                           header="true")

### **Display the first <code>n=5</code> rows of the loaded dataset**

In [17]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(wine_df.count(), len(wine_df.columns)))

The shape of the dataset is 129975 rows by 14 columns


In [18]:
wine_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [15]:
wine_df.show(n=10, truncate=False)

+---+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+-----------------------------------------------------------------------------------+------------------+-------------------+
|_c0|country |description                                                                                                                                                                                                                                                          |designation                       |points|price|province         |region_1           |region_2         |taster_name       |taster_twitter_handle|title                             

# **Cleaing the data set**

Count the number of duplicated description (if any)

In [19]:
print("The total number of duplicated description are {:d} out of {:d}".
      format(wine_df.count() - wine_df.dropDuplicates(['description']).count(), wine_df.count()))

The total number of duplicated description are 10016 out of 129975


In [22]:
wine_df.groupby(["description"]).count().sort("count", ascending=False).show(10,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|description                                                                                                                                                                                                                                                                                                  |count|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Stalky aromas suggest hay and green herbs, with raspberry in the back

### **Remove duplicate description**

In [24]:
wine_df = wine_df.dropDuplicates(["description"])

In [25]:
print("The total number of unique description is: {:d}".format(wine_df.count()))

The total number of unique description is: 119959


### Check for any missing value (i.e., <code>NULL</code>) along <code>description</code> column  and 

In [27]:
wine_df.where(col("description").isNull()).count()

1

In [28]:
wine_df.where(col("variety").isNull()).count()

11

### Removing the rows that have null description and variety

In [31]:
wine_df = wine_df.na.drop(subset=["variety"])

In [32]:
wine_df.where(col("variety").isNull()).count()

0

In [33]:
wine_df = wine_df.na.drop(subset=["description"])

In [34]:
wine_df.where(col("description").isNull()).count()

0

In [35]:
print("The total number of unique description is: {:d}".format(wine_df.count()))

The total number of unique description is: 119947
